In [268]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import datetime
import pandas as pd
import time
import re
from dateutil.relativedata import relativedelta

ModuleNotFoundError: No module named 'dateutil.relativedata'

Set date range to be the past 365 days.

In [267]:
end = datetime.date.today() 
start = end - relativedelta(years=1)
print(start)

TypeError: 'module' object is not callable

In [261]:
start = datetime.date(2019, 12, 8)
end = datetime.date.today() 
date_range = pd.date_range(start, periods=(end-start).days).tolist()
print('Range: ' + str(start) + ' to ' + str(end)) 
print('Length: ' + str(num_days) + ' days')
date_range[:10]

Range: 2019-12-08 to 2020-04-19
Length: 132 days


[Timestamp('2019-12-08 00:00:00', freq='D'),
 Timestamp('2019-12-09 00:00:00', freq='D'),
 Timestamp('2019-12-10 00:00:00', freq='D'),
 Timestamp('2019-12-11 00:00:00', freq='D'),
 Timestamp('2019-12-12 00:00:00', freq='D'),
 Timestamp('2019-12-13 00:00:00', freq='D'),
 Timestamp('2019-12-14 00:00:00', freq='D'),
 Timestamp('2019-12-15 00:00:00', freq='D'),
 Timestamp('2019-12-16 00:00:00', freq='D'),
 Timestamp('2019-12-17 00:00:00', freq='D')]

In [12]:
date_range_for_plots = [str(x).split(' ')[0] for x in date_range]
date_range_for_plots[0]

'2019-12-08'

Get the New York Times link for each day (it will give us the first snapshot).

In [64]:
def get_links(url):
    browser.get(url)
    calendar_grid = WebDriverWait(browser, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, 'calendar-grid'))) 
    if '2019' in url:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(5)
    dates = calendar_grid.find_elements_by_tag_name('a') 
    return [date.get_attribute('href') for date in dates]

def scrape_archives(urls):
    browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver') 
    browser.maximize_window() 
    data = {}
    for url in urls: 
        links = get_links(url)
        for link in links: 
            date = datetime.datetime.strptime(re.search('web/(\d*)', link)[1], '%Y%m%d')
            if date in date_range:
                data[date] = {'link': link,
                              'headlines': [],
                              'sections': []}
    browser.quit()
    
urls = ['http://web.archive.org/web/2019*/https://www.nytimes.com/', 
        'http://web.archive.org/web/*/nytimes.com']

scrape_archives(urls)

In [251]:
# For each date, the previous day's link gets that day (idk why)
data2 = {}

for date in date_range:
    date_str = (date - timedelta(days=1)).strftime('%Y%m%d')
    link = 'http://web.archive.org/web/' + date_str + '/https://www.nytimes.com/'
    data2[date] = {'link': link,
                   'headlines': [],
                   'sections': []}

In [258]:
data2

{Timestamp('2019-12-08 00:00:00', freq='D'): {'link': 'http://web.archive.org/web/20191207/https://www.nytimes.com/',
  'headlines': ['Video Games and Online Chats Are ‘Hunting Grounds’ for Sexual Predators',
   'Here’s how to protect your children.',
   'Florida Shooting Suspect Showed Videos of Mass Shootings at Party',
   'After two attacks on Navy bases in a week, officials are confronting how persistent such incidents have become.',
   'Judiciary Committee Releases Report Defining Impeachable Offense',
   'Behind the Scenes of Impeachment: Crammed Offices, Late Nights, Cold Pizza',
   'Buttigieg Struggles to Square Transparency With Lack of Disclosure on Consulting',
   'As Candidates Jostle for Position, a Long Race May Become a Marathon',
   'Can Biology Class Reduce Racism?',
   '11 of Our Best Weekend Reads',
   'Did you stay up-to-date this week? Take our news quiz.',
   'Listen: ‘Modern Love’ Podcast',
   'The ‘In Her Words’ Newsletter',
   'The Neediest Cases Fund',
   'The

Get headlines and sections from each link.

In [255]:
browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver') 
unacceptable_headlines = ['Listen to ‘The Daily’']

for date in data2:

    print('Scraping ' + str(date) + '...')
    browser.get(data2[date]['link'])
    time.sleep(5)
    body = WebDriverWait(browser, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, 'e6b6cmu0'))) 
    
    elements = [x for x in body.find_elements_by_class_name('esl82me0') if x.text not in unacceptable_headlines]

    # Parse headlines
    data2[date]['headlines'] = [headline.text for headline in elements]

    # Parse sections (anchor elements contain headlines)
    links = []
    for x in elements:
        link = x.find_element_by_xpath('../..').get_attribute('href')
        if not link:
            link = x.find_element_by_xpath('..').get_attribute('href')
        if not link:
            link = x.find_element_by_xpath('../../..').get_attribute('href')
        links.append(link)

    sections = []
    for x in links: 
        if 'cooking.nytimes.com' in x:
            section = 'cooking'
        elif '/us/' and '/politics/' in x:
            section = 'us-politics'
        elif 'www.nytimes.com/weekender' in x:
            section = 'weekender'
        elif 'www.nytimes.com/live' in x:
            section = 'live'
        elif 'www.nytimes.com/spotlight' in x:
            section = 'spotlight'
        else:
            matches = re.search('/(?:\d{2,4}/){1,3}([\w-]*)|nytimes.com/(\D*)/', x)
            if matches[1]:
                section = matches[1]
            else:
                section = matches[2]
        sections.append(section)

    data2[date]['sections'] = sections
    print('Found ' + str(len(elements)) + ' headlines') 

Scraping 2019-12-08 00:00:00...
Found 28 headlines
Scraping 2019-12-09 00:00:00...
Found 26 headlines
Scraping 2019-12-10 00:00:00...
Found 30 headlines
Scraping 2019-12-11 00:00:00...
Found 29 headlines
Scraping 2019-12-12 00:00:00...
Found 27 headlines
Scraping 2019-12-13 00:00:00...
Found 28 headlines
Scraping 2019-12-14 00:00:00...
Found 27 headlines
Scraping 2019-12-15 00:00:00...
Found 28 headlines
Scraping 2019-12-16 00:00:00...
Found 26 headlines
Scraping 2019-12-17 00:00:00...
Found 25 headlines
Scraping 2019-12-18 00:00:00...
Found 28 headlines
Scraping 2019-12-19 00:00:00...
Found 31 headlines
Scraping 2019-12-20 00:00:00...
Found 28 headlines
Scraping 2019-12-21 00:00:00...
Found 27 headlines
Scraping 2019-12-22 00:00:00...
Found 28 headlines
Scraping 2019-12-23 00:00:00...
Found 26 headlines
Scraping 2019-12-24 00:00:00...
Found 24 headlines
Scraping 2019-12-25 00:00:00...
Found 26 headlines
Scraping 2019-12-26 00:00:00...
Found 25 headlines
Scraping 2019-12-27 00:00:00...

In [257]:
data2

{Timestamp('2019-12-08 00:00:00', freq='D'): {'link': 'http://web.archive.org/web/20191207/https://www.nytimes.com/',
  'headlines': ['Video Games and Online Chats Are ‘Hunting Grounds’ for Sexual Predators',
   'Here’s how to protect your children.',
   'Florida Shooting Suspect Showed Videos of Mass Shootings at Party',
   'After two attacks on Navy bases in a week, officials are confronting how persistent such incidents have become.',
   'Judiciary Committee Releases Report Defining Impeachable Offense',
   'Behind the Scenes of Impeachment: Crammed Offices, Late Nights, Cold Pizza',
   'Buttigieg Struggles to Square Transparency With Lack of Disclosure on Consulting',
   'As Candidates Jostle for Position, a Long Race May Become a Marathon',
   'Can Biology Class Reduce Racism?',
   '11 of Our Best Weekend Reads',
   'Did you stay up-to-date this week? Take our news quiz.',
   'Listen: ‘Modern Love’ Podcast',
   'The ‘In Her Words’ Newsletter',
   'The Neediest Cases Fund',
   'The

In [225]:
# Test
for element in elements:
    print(element.text)
    link = element.find_element_by_xpath('../..').get_attribute('href')
    link2 = element.find_element_by_xpath('..').get_attribute('href')
    link3 = element.find_element_by_xpath('../../..').get_attribute('href')
    print('link 1: ' + str(link))
    print('link 2: ' + str(link2))
    print('link 3: ' + str(link3))

Trump Signs $2 Trillion Coronavirus Relief Package
link 1: http://web.archive.org/web/20200327235926/https://www.nytimes.com/2020/03/27/world/coronavirus-news.html
link 2: None
link 3: None
Updates: Measure Is Largest Stimulus in Modern History
link 1: http://web.archive.org/web/20200327235926/https://www.nytimes.com/2020/03/27/world/coronavirus-news.html
link 2: None
link 3: None
Updates: New York Region
link 1: http://web.archive.org/web/20200327235926/https://www.nytimes.com/2020/03/27/nyregion/coronavirus-new-york-update.html
link 2: None
link 3: None
Some U.S. Cities Could Have Outbreaks Worse Than Wuhan’s
link 1: http://web.archive.org/web/20200327235926/https://www.nytimes.com/interactive/2020/03/27/upshot/coronavirus-new-york-comparison.html
link 2: None
link 3: None
Updates: Business and Markets
link 1: http://web.archive.org/web/20200327235926/https://www.nytimes.com/2020/03/27/business/stock-market-today-coronavirus.html
link 2: None
link 3: None
‘White-Collar Quarantine’: V

In [232]:
for i in elements:
    if i.text == 'Listen to ‘The Daily’':
        print(i)

<selenium.webdriver.remote.webelement.WebElement (session="bac102a63ca05026b1c33d33b511cba0", element="de5c11e7-124c-498c-a4cb-754f95db04f2")>


In [242]:
data[datetime.datetime(2020, 3, 27, 0)]

{'link': 'http://web.archive.org/web/20200327/nytimes.com',
 'headlines': ['Trump Signs $2 Trillion Coronavirus Relief Package',
  'Updates: Measure Is Largest Stimulus in Modern History',
  'Updates: New York Region',
  'Some U.S. Cities Could Have Outbreaks Worse Than Wuhan’s',
  'Updates: Business and Markets',
  '‘White-Collar Quarantine’: Virus Puts Spotlight on Class Divide',
  'The Times is providing free access on the coronavirus crisis. Start here for a guide to all our coverage.',
  'Your Friday Evening Briefing',
  'Listen to ‘The Daily’',
  'Listen to ‘The Argument’',
  'New Rochelle, Once a Coronavirus Hot Spot, May Now Offer Hope',
  'America’s Middle Braces for Onslaught as Virus Moves Beyond Coasts',
  'Where the U.S. Stands Now on Coronavirus Testing',
  '‘The Other Option Is Death’: New York Starts Sharing of Ventilators',
  'Cardiologists are seeing infected patients whose worst symptoms are not respiratory, but cardiac.',
  '‘We Take the Dead From Morning Till Night

2019-12-08 00:00:00
2019-12-09 00:00:00
2019-12-10 00:00:00
2019-12-11 00:00:00
2019-12-12 00:00:00
2019-12-13 00:00:00
2019-12-14 00:00:00
2019-12-15 00:00:00
2019-12-16 00:00:00
2019-12-17 00:00:00
2019-12-18 00:00:00
2019-12-19 00:00:00
2019-12-20 00:00:00
2019-12-21 00:00:00
2019-12-22 00:00:00
2019-12-23 00:00:00
2019-12-24 00:00:00
2019-12-25 00:00:00
2019-12-26 00:00:00
2019-12-27 00:00:00
2019-12-28 00:00:00
2019-12-29 00:00:00
2019-12-30 00:00:00
2019-12-31 00:00:00
2020-01-01 00:00:00
2020-01-02 00:00:00
2020-01-03 00:00:00
2020-01-04 00:00:00
2020-01-05 00:00:00
2020-01-06 00:00:00
2020-01-07 00:00:00
2020-01-08 00:00:00
2020-01-09 00:00:00
2020-01-10 00:00:00
2020-01-11 00:00:00
2020-01-12 00:00:00
2020-01-13 00:00:00
2020-01-14 00:00:00
2020-01-15 00:00:00
2020-01-16 00:00:00
2020-01-17 00:00:00
2020-01-18 00:00:00
2020-01-19 00:00:00
2020-01-20 00:00:00
2020-01-21 00:00:00
2020-01-22 00:00:00
2020-01-23 00:00:00
2020-01-24 00:00:00
2020-01-25 00:00:00
2020-01-26 00:00:00
